### Memory mit Flaggen auf Canvas

In [ ]:
import memory as M

In [ ]:
M.init(2*list('ABCD'))
M.set_callback(print)    

In [ ]:
layout = M.state['layout']
layout

In [ ]:
matches = [(i,j) for i,x in layout.items() for j,y in layout.items() if i < j and x == y]
matches

In [ ]:
picks1 = matches[:2] + [(matches[2][0], matches[3][1])]
picks2 = matches[2:] 
picks1, picks2

In [ ]:
for pick in picks1:
    M.face_up(*pick)
    M.face_down()

In [ ]:
for pick in picks2:
    M.face_up(*pick)
    M.face_down()

***
Wir schreiben nun eine Funktion `M.callback`, so dass der Spielzustand in einem Canvas-Widget dargestellt wird.
***

In [ ]:
from ipycanvas import MultiCanvas
from draw_flags import draw_CH_flag, draw_BRD_flag, draw_BE_flag

***
Unser Deck besteht nun nicht mehr aus Buchstaben, sondern aus Funktionen.
***

In [ ]:
# vgl. deck = list('ABC') * 2
flags = [draw_CH_flag, draw_BRD_flag, draw_BE_flag] * 2
# flags[0]
flags[0].__name__

In [ ]:
FLAG_SIZE = 100
width, height = (3 * FLAG_SIZE, 2 * FLAG_SIZE)
mcanvas = MultiCanvas(2, width = width, height = height, 
                layout = {'border' : '2px solid black'}
               )

bg, fg = mcanvas
fg.line_width = 2
fg.stroke_style = 'black'
bg.fill_style = 'blue'

# horizonale Linie
fg.stroke_lines([(0, height/2), (width, height/2)])
# vertikale Linien
for i in range(1, 3):
    fg.stroke_lines([(i*width/3, 0), (i*width/3, height)])

display(mcanvas)

In [ ]:
# Teste Funktionen, zeichne Flaggen auf bg
for i, flag in enumerate(flags):
    x = (i % 3)  * FLAG_SIZE
    y = (i // 3) * FLAG_SIZE
    flag(bg, (x, y), FLAG_SIZE)

In [ ]:
bg.clear()

In [ ]:
def handle_event(canvas, event, data = None):
    
    def get_position(idx):
        return (FLAG_SIZE * (idx % 3), FLAG_SIZE * (idx // 3))
    
    canvas.fill_style = 'blue'
    
    if event == 'new_game':
        canvas.clear()
        for pos in [get_position(i) for i in range(6)]:
            canvas.fill_rect(*pos, FLAG_SIZE)
            
    if event == 'phase_1':
        (idx1, flag1), (idx2, flag2) = data
        pos1 = get_position(idx1)
        pos2 = get_position(idx2) 
        flag1(canvas, pos1, FLAG_SIZE)
        flag2(canvas, pos2, FLAG_SIZE)
    
    if event == 'phase_2':
        match, idxs = data
        positions = [get_position(idx) for idx in idxs]
        for pos in positions:
            canvas.clear_rect(*pos, FLAG_SIZE)
            
        if not match:
            for pos in positions:
                canvas.fill_rect(*pos, FLAG_SIZE)

In [ ]:
handle_event(bg, 'new_game')

In [ ]:
picks = ((1, flags[1]), (3, flags[3]))
handle_event(bg, 'phase_1', picks)

In [ ]:
handle_event(bg, 'phase_2', (True, (1,3),))

In [ ]:
handle_event(bg, 'phase_2', (False, (1,3),))

***
**Play Memory**
***

In [ ]:
bg.clear()
display(mcanvas)

In [ ]:
callback = lambda event, data: handle_event(bg, event, data)
M.set_callback(callback)
M.init(flags)

In [ ]:
layout = M.state['layout']
matches = [(i,j) for i,x in layout.items() for j,y in layout.items() if i < j and x == y]
(a1, a2), (b1, b2), (c1, c2) = matches
matches

In [ ]:
M.face_up(a1, a2)

In [ ]:
M.face_down()

In [ ]:
M.face_up(b1, c1)

In [ ]:
M.face_down()

In [ ]:
M.face_up(b1, b2)

In [ ]:
M.face_down()

In [ ]:
M.face_up(c1, c2)

In [ ]:
M.face_down()

In [ ]:
# wer hat was gesammelt?
{player:[m.__name__ for m in matches] for player, matches in M.state['matches'].items()}